In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
import numpy as np

In [4]:
input_ = torch.randn(1, 3, 128, 128)
model = models.resnet18()

## 取出特定層資訊

### 方法1 : 取出子 modules

In [5]:
for name,_ in model.named_children():
    print(name)

conv1
bn1
relu
maxpool
layer1
layer2
layer3
layer4
avgpool
fc


In [6]:
## 假設我們只需要 convolution最終的 output == 不需要 pooling, fc
sub_model = nn.Sequential(*list(model.children())[:-2])

In [7]:
input_ = torch.randn(1, 3, 128, 128)
output = sub_model(input_)
print('convolution 層 output shape : {}'.format(output.shape))

convolution 層 output shape : torch.Size([1, 512, 4, 4])


D:\ProgramData\Anaconda3\envs\py39\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


### 接上自定義 output layer

In [8]:
class MyModel(nn.Module):
    def __init__(self, output_class=5):
        super(MyModel, self).__init__()
        self.backbone = sub_model
        self.output = nn.Linear(512 * 4 * 4, output_class)
    
    
    \
    def forward(self, x):
        x = self.backbone(x)
        batch_size = x.shape[0]
        x = x.reshape(batch_size, -1)
        x = self.output(x)
        return(x)

In [9]:
updated_model = MyModel()

In [10]:
input_ = torch.randn(1, 3, 128, 128)
output = updated_model(input_)
print('output shape : {}'.format(output.shape))

output shape : torch.Size([1, 5])


### 自行定義 forward

In [11]:
class MyModel(nn.Module):
    def __init__(self, output_class=5):
        super(MyModel, self).__init__()
        self.input_cov = model.conv1
        self.first_bn = model.bn1
        self.first_relu = model.relu
        self.maxpool = model.maxpool
        self.layer1 = model.layer1
        self.layer2 = model.layer2
        self.layer3 = model.layer3
        self.layer4 = model.layer4
        self.avpool = model.avgpool
        self.output = nn.Linear(512, output_class)
    def forward(self, x):
        x = self.input_cov(x)
        x = self.first_bn(x)
        x = self.first_relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avpool(x)
        
        x = x.reshape(-1, 512)
        x = self.output(x)
        
        return(x)

In [12]:
new_model = MyModel()

In [13]:
input_ = torch.randn(1, 3, 128, 128)
output = new_model(input_)
print('output shape : {}'.format(output.shape))

output shape : torch.Size([1, 5])


### Hook
* 延伸閱讀：https://zhuanlan.zhihu.com/p/75054200

#### register_forward_hook, register_backward_hook
* hook直譯為鉤子，可以理解為鉤住我們想要的資訊。
* .forward , .backward 是 nn.Module 底下的函數，而register_forward_hook, register_backward_hook顧名思義就是針對兩個函數自訂欲取出的內容。
* 每當我們執行 model.forward(), model.backward() 被註冊的 hook就會啟動。

In [14]:
# register_forward_hook
# register_backward_hook

In [15]:
input_ = torch.randn(1, 3, 128, 128)
model = models.resnet18()

In [16]:
## 假設我們想要取出 layer1 第一個BasicBlock的 output
model.children

<bound method Module.children of ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  

In [17]:
outputs= []
def layer1_hook(module, input_, output):
    outputs.append(output)
    
model.layer1[0].register_forward_hook(layer1_hook)

In [18]:
outputs

[]

In [19]:
outupt = model(input_)

In [20]:
outputs[0].shape

torch.Size([1, 64, 32, 32])

### hook延伸應用：假設我們想看到每一個子 module 輸出tensor的shape

In [21]:
input_ = torch.randn(1, 3, 128, 128)
model = models.resnet18()

In [22]:
outputs= []
def hook(module, input_, output):
    outputs.append(output.shape)

for module in model.children():
    module.register_forward_hook(hook)


In [23]:
outupt = model(input_)

In [24]:
outputs

[torch.Size([1, 64, 64, 64]),
 torch.Size([1, 64, 64, 64]),
 torch.Size([1, 64, 64, 64]),
 torch.Size([1, 64, 32, 32]),
 torch.Size([1, 64, 32, 32]),
 torch.Size([1, 128, 16, 16]),
 torch.Size([1, 256, 8, 8]),
 torch.Size([1, 512, 4, 4]),
 torch.Size([1, 512, 1, 1]),
 torch.Size([1, 1000])]

### Weight initialization
* 預設：https://github.com/pytorch/pytorch/tree/master/torch/nn/modules
* 參考每個 function底下的 : reset_parameters
* PyTorch Initialization function：https://pytorch.org/docs/stable/nn.init.html

In [49]:
m = nn.Conv2d(3, 6, 5)

In [25]:
from torch.nn import init

def weights_init(m):
    #classname = m.__class__.__name__
    if isinstance(m, nn.Conv2d):
        #torch.nn.init.xavier_uniform_(m.weight)
        nn.init.kaiming_normal_(m.weight.data,
                                    a=0,
                                    mode='fan_out',
                                    nonlinearity='relu')
        if m.bias is not None:
            torch.nn.init.zeros_(m.bias)
    elif isinstance(m, nn.Linear):
        n = m.in_features
        y = 1.0/np.sqrt(n)
        m.weight.data.uniform_(-y, y)
        if m.bias is not None:
            m.bias.data.fill_(0)


model.apply(weights_init)
print('init success...')

init success...


### 得到模型完整資訊

In [26]:
#pip install torchsummary
from torchsummary import summary

In [27]:
input_ = torch.randn(1, 3, 128, 128)
model = models.resnet18()

In [28]:
summary(model, input_,)

Layer (type:depth-idx)                   Output Shape              Param #
├─Conv2d: 1-1                            [-1, 64, 64, 64]          9,408
├─BatchNorm2d: 1-2                       [-1, 64, 64, 64]          128
├─ReLU: 1-3                              [-1, 64, 64, 64]          --
├─MaxPool2d: 1-4                         [-1, 64, 32, 32]          --
├─Sequential: 1-5                        [-1, 64, 32, 32]          --
|    └─BasicBlock: 2-1                   [-1, 64, 32, 32]          --
|    |    └─Conv2d: 3-1                  [-1, 64, 32, 32]          36,864
|    |    └─BatchNorm2d: 3-2             [-1, 64, 32, 32]          128
|    |    └─ReLU: 3-3                    [-1, 64, 32, 32]          --
|    |    └─Conv2d: 3-4                  [-1, 64, 32, 32]          36,864
|    |    └─BatchNorm2d: 3-5             [-1, 64, 32, 32]          128
|    |    └─ReLU: 3-6                    [-1, 64, 32, 32]          --
|    └─BasicBlock: 2-2                   [-1, 64, 32, 32]          --
|

Layer (type:depth-idx)                   Output Shape              Param #
├─Conv2d: 1-1                            [-1, 64, 64, 64]          9,408
├─BatchNorm2d: 1-2                       [-1, 64, 64, 64]          128
├─ReLU: 1-3                              [-1, 64, 64, 64]          --
├─MaxPool2d: 1-4                         [-1, 64, 32, 32]          --
├─Sequential: 1-5                        [-1, 64, 32, 32]          --
|    └─BasicBlock: 2-1                   [-1, 64, 32, 32]          --
|    |    └─Conv2d: 3-1                  [-1, 64, 32, 32]          36,864
|    |    └─BatchNorm2d: 3-2             [-1, 64, 32, 32]          128
|    |    └─ReLU: 3-3                    [-1, 64, 32, 32]          --
|    |    └─Conv2d: 3-4                  [-1, 64, 32, 32]          36,864
|    |    └─BatchNorm2d: 3-5             [-1, 64, 32, 32]          128
|    |    └─ReLU: 3-6                    [-1, 64, 32, 32]          --
|    └─BasicBlock: 2-2                   [-1, 64, 32, 32]          --
|